In [8]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("/Users/henryfetzer/Documents/ACME/Vol3FallProj/data/Life_Expectancy_Data.csv")

In [3]:
print(df.shape)
df = df.dropna()
df = df.rename(columns={'Life expectancy ': 'Life expectancy', 'under-five deaths ':'under-five deaths',
                        'Measles ':'Measles'})
df.drop(columns=['Country', 'Status'], inplace=True)
print(df.shape)


(2938, 22)
(1649, 20)


In [4]:
# Create target column
df['life_over_70'] = (df['Life expectancy'] >= 70).astype(int)

In [5]:
# Train test split
data = df.drop(columns=['life_over_70', 'Life expectancy', 'Adult Mortality'])
target = df['life_over_70']
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

print(X_train.shape)

(1319, 18)


In [8]:
#Grid search for best classifier
# classifier
gbc = GradientBoostingClassifier()

# specify hyperparameters
param_grid = {"alpha": ['log_loss', 'exponential'],
              "gamma": [3,5],
              "lambda": [1,3],
              "eta": [100,200]}
gbc_gs = GridSearchCV(gbc, param_grid, scoring="f1", n_jobs=-1, verbose=5, cv=2)

# run the search
gbc_gs.fit(X_train, y_train)


# print best parameters and resulting score
print(gbc_gs.best_params_, gbc_gs.best_score_, sep='\n')

Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV 1/2] END loss=log_loss, max_depth=3, min_samples_leaf=1, n_estimators=100;, score=0.930 total time=   0.7s
[CV 2/2] END loss=log_loss, max_depth=3, min_samples_leaf=1, n_estimators=100;, score=0.957 total time=   0.7s
[CV 1/2] END loss=log_loss, max_depth=3, min_samples_leaf=1, n_estimators=200;, score=0.934 total time=   1.6s
[CV 1/2] END loss=log_loss, max_depth=3, min_samples_leaf=3, n_estimators=100;, score=0.929 total time=   0.9s
[CV 2/2] END loss=log_loss, max_depth=3, min_samples_leaf=1, n_estimators=200;, score=0.962 total time=   1.7s
[CV 2/2] END loss=log_loss, max_depth=3, min_samples_leaf=3, n_estimators=100;, score=0.950 total time=   0.9s
[CV 2/2] END loss=log_loss, max_depth=5, min_samples_leaf=1, n_estimators=100;, score=0.966 total time=   1.5s
[CV 1/2] END loss=log_loss, max_depth=5, min_samples_leaf=1, n_estimators=100;, score=0.951 total time=   1.5s
[CV 1/2] END loss=log_loss, max_depth=3, min_sample

In [9]:
# zip features importance and names together
feature_names = data.columns
features_and_importance = sorted(zip(gbc_gs.best_estimator_.feature_importances_, feature_names), 
                                 reverse=True)

for importance, name in features_and_importance:
    print(f"{name}: {importance}")

Income composition of resources: 0.556288852383641
 thinness 5-9 years: 0.07111483719764868
 HIV/AIDS: 0.06033280162929234
Alcohol: 0.060010205360026186
 thinness  1-19 years: 0.053797069613392166
Total expenditure: 0.041781470720396986
under-five deaths: 0.025484754073220903
infant deaths: 0.02520890565369538
GDP: 0.02170682221624766
Schooling: 0.018980604592068986
 BMI : 0.016746971683107112
Year: 0.01347590212881509
Hepatitis B: 0.010546794733175723
Population: 0.006175538745162405
Diphtheria : 0.005494308146426417
Polio: 0.004608874539640368
percentage expenditure: 0.004287169550525578
Measles: 0.00395811703351698


The results look similar to those for the random forest. We don't yet have a higher level of accuracy.